# Praca Domowa 5
Bartosz Siński

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
np.set_seed = 42

### Wczytanie danych

In [ ]:
X = pd.read_csv("./src/clustering.csv")

In [ ]:
X

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(X.iloc[:,0], X.iloc[:,1])
plt.show()

### Metoda k-średnich

Znajdziemy najbardziej optymalną liczbę klastrów z użyciem metody Silhouette.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
ss = []
k = range(2, 15)
for i in k:
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    labels = kmeans.predict(X)
    score = silhouette_score(X,labels, random_state=42)
    ss.append(score)
plt.plot(k,ss,'bo-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.show()

Widzimy, że najwyższa wartość Silhouette Score jest dla k=8. Zobaczmy jak wygląda wizualizacja przypisania naszych danych do ośmiu klastrów metodą k-średnich.

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(X)
preds = kmeans.predict(X)
centers = kmeans.cluster_centers_
plt.figure(figsize=(6,6))
plt.scatter(X.iloc[:,0],X.iloc[:,1],c=preds,cmap="Set1")
plt.scatter(centers[:,0],centers[:,1],s=100,c="black")
plt.show()

### Hierarchiczna metoda aglomeracyjna z połączeniami Warda

Na poczatku znajdziemy optymalną liczbe klastrów przy użyciu mertyki Dunn Index. Poniższa implementacja pochodzi z https://github.com/jqmviegas/jqm_cvi/blob/master/jqmcvi/base.py. Wylicza ona indeks na podstawie najmniejszej odległości między punktami najbliższych klastrów i najwiekszej odległości pomiędzy klastrami

In [ ]:
def delta(ck, cl):
    values = np.ones([len(ck), len(cl)])*10000
    
    for i in range(0, len(ck)):
        for j in range(0, len(cl)):
            values[i, j] = np.linalg.norm(ck[i]-cl[j])

    return np.min(values)
    
def big_delta(ci):
    values = np.zeros([len(ci), len(ci)])
    
    for i in range(0, len(ci)):
        for j in range(0, len(ci)):
            values[i, j] = np.linalg.norm(ci[i]-ci[j])
            
    return np.max(values)
    
def dunn(k_list):
    deltas = np.ones([len(k_list), len(k_list)])*1000000
    big_deltas = np.zeros([len(k_list), 1])
    l_range = list(range(0, len(k_list)))
    
    for k in l_range:
        for l in (l_range[0:k]+l_range[k+1:]):
            deltas[k, l] = delta(k_list[k], k_list[l])
        
        big_deltas[k] = big_delta(k_list[k])

    di = np.min(deltas)/np.max(big_deltas)
    return di

In [ ]:
from sklearn.cluster import AgglomerativeClustering
scores = []
k = range(2,20)
for i in k:
    labels = pd.DataFrame(AgglomerativeClustering(n_clusters=i).fit_predict(X))
    labels.columns=["Label"]
    pred = pd.concat([X,labels],axis=1)
    clusters = []
    for j in range(0,i):
        obs = pred.loc[pred.Label==j]
        clusters.append(obs.iloc[:,[0,1]].values)
    scores.append(dunn(clusters))
plt.plot(k,scores,'bo-')
plt.xlabel('k')
plt.ylabel('Dunn Index')
plt.show()

Najwyższe wartości indeksu Dunna są dla k = [6, 7, 8]. Zobaczymy jak będą wyglądać klastry naszych danych.

In [ ]:
fig, axs = plt.subplots(nrows=3,figsize=(5,15))
pred1 = AgglomerativeClustering(n_clusters=6).fit_predict(X)
pred2 = AgglomerativeClustering(n_clusters=7).fit_predict(X)
pred3 = AgglomerativeClustering(n_clusters=8).fit_predict(X)
axs[0].scatter(X.iloc[:,0],X.iloc[:,1],c=pred1,cmap="Set1")
axs[0].set_title("k = 6")
axs[1].scatter(X.iloc[:,0],X.iloc[:,1],c=pred2,cmap="Set1")
axs[1].set_title("k = 7")
axs[2].scatter(X.iloc[:,0],X.iloc[:,1],c=pred3,cmap="Set1")
axs[2].set_title("k = 8")
plt.show()